# Introduction

This notebook provides simple implementations of the sigma protocols specified [here](https://github.com/zkpstandard/wg-sigma-protocols/)

In [119]:
from hashlib import sha3_256 as hash_function
from secrets import token_bytes as random_bytes
import pickle

DOMSEP = b"zkpstd/sigma/0.1"

HD = hash_function()

BLOCK_LEN = HD.block_size
DIGEST_LEN = HD.digest_size

HD.update(DOMSEP)
HD = HD.digest() 

'''
Pads bytestring `s` with 0's, so that its length is a multiple of `BLOCK_LEN`
'''
def pad_to_blocklen(s: bytes) -> bytes: 
    padding = b"\000" * (BLOCK_LEN - (len(s) % BLOCK_LEN))
    return s + padding

'''

'''
def hash_to_seed(b: bytes) -> int:
    return int.from_bytes(b[:4], "big")

* `prover_commit` generates a commitment `T` and a prover state `pstate`. The prover commits to the secret witness in a way that doesn't reveal anything about the witness itself. Commitment `T` is sent to the verifier, and `pstate` is kept secret.
* `prover_response` provides a proof of knowledge given a challenge and a prover state.
* `label` describes the statement, parameters, and relation being proven in a 32 byte string. It's used for computing the Fiat-Shamir transform. 
* `simulate_commitment` computes a uniformly random resonse from the same distribution as `prover_commit`, given challenge and response. 
* `simulate_response` computes a uniformly random response from the same distribution as `prover_response`.

In [173]:
class SigmaProtocol:
    def prover_commit(self, witness):
        pass

    def prover_response(self, state, challenge):
        pass
      
    def label(self):
        pass
    
    def simulate_commitment(self, challenge, response):
        pass
    
    def simulate_response(self):
        pass
    
    def challenge(self, message: bytes, commitment) -> bytes:
        hm = hash_function()
        hm.update(message)
        hm = pad_to_blocklen(hm.digest())
        
        commitment_bytes = pad_to_blocklen(pickle.dumps(commitment))
        
        result = self.first_block_hash.copy()
        result.update(hm)
        result.update(commitment_bytes)
        return result.digest()
    
    def _get_first_block_hash(self, ctx: bytes):
        hctx = hash_function()
        hctx.update(ctx)
        hctx = hctx.digest()
        
        first_block_hash = hash_function()
        first_block_hash.update(pad_to_blocklen(HD + self.label() + hctx))
        return first_block_hash
    
    def verifier(self, commitment, challenge, response):
        pass        
    
    def batchable_proof(self, witness, message: bytes):
        pstate, commitment = self.prover_commit(witness)
        challenge = self.challenge(message, commitment)
        response = self.prover_response(pstate, challenge)
        return (pickle.dumps(commitment), pickle.dumps(response))
    
    def batchable_verify(self, proof, message: bytes):
        commitment_bytes, response_bytes = proof
        commitment = pickle.loads(commitment_bytes)
        response = pickle.loads(response_bytes)
        
        challenge = self.challenge(message, commitment)
        return self.verifier(commitment, challenge, response)
    
    def short_proof(self, witness, message: bytes):
        pstate, commitment = self.prover_commit(witness)
        challenge = self.challenge(message, commitment)
        response = self.prover_response(pstate, challenge)
        return (pickle.dumps(challenge), pickle.dumps(response))
    
    def short_verify(self, proof, message: bytes):
        challenge_bytes, response_bytes = proof
        challenge = pickle.loads(challenge_bytes)
        response = pickle.loads(response_bytes)
        
        commitment = self.simulate_commitment(challenge, response)
        challenge_prime = self.challenge(message, commitment)
        return challenge == challenge_prime

These are parameters for `secp256k1`, one of our standard's supported curves. `Zp` is the scalar field, and `G` is the prime-order group of points.

I got the constants from [here.](https://medium.com/asecuritysite-when-bob-met-alice/ni-zkp-using-discrete-log-equality-dleq-with-secp256k1-p-256-curve-25519-and-curve-448-c28ab7044259)

In [174]:
from collections import namedtuple

Zq = GF(0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f)
E = EllipticCurve(Zq, [0, 7])
G = E.lift_x(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798)
H = E.lift_x(0xc9e777de60dff3bb651b89183754bbc633c34b124429afe8cbaff130abb7bff5)
p = G.order()
Zp = GF(p)

EC = namedtuple('EC', ['G', 'H', 'E', 'p', 'Fp'])
secp256k1 = EC(G, H, E, p, Zp)

In [221]:
class DlogTemplate(SigmaProtocol):
    # n is the input dimension of the homomorphism
    # m is the output dimension of the homomorphism
    n = None
    m = None
    
    # ec specifies the elliptic curve used
    # defaults to secp256k1
    ec = secp256k1
    
    def __init__(self, ctx: bytes, statement):
        assert(len(statement) == self.n)
        self.statement = statement
        self.first_block_hash = self._get_first_block_hash(ctx)
        
    def prover_commit(self, witness):
        assert(len(witness) == self.m)
        
        # First, seed rng
        witness_bytes = pickle.dumps(witness)
        second_block = pad_to_blocklen(random_bytes(32) + witness_bytes)
        h = self.first_block_hash.copy()
        h.update(second_block)
        commit_seed = hash_to_seed(h.digest())
        with seed(commit_seed):
            # nonce is an n-length array of Fp elements
            nonce = [self.ec.Fp.random_element() for _ in range(self.n)]
            # commitment is an m-length array of G2 elements
            commitment = self._morphism(nonce)
            prover_state = (witness, nonce)
            return (prover_state, commitment)
    
    # Prover_state is nonce (array of n Fp elements) * witness (array of n Fp elements)
    # challenge is bytes, but is converted to an Fp element
    # outputs an array of n Fp elements
    def prover_response(self, prover_state, challenge: bytes):
        witness, nonce = prover_state
        challenge = self._chal_from_bytes(challenge)
        return [challenge * witness_i + nonce_i 
                for (witness_i, nonce_i) in zip(witness, nonce)]
    
    def simulate_response(self):
        return [self.ec.Fp.random_element() 
                for _ in range(self.m)]
    
    def simulate_commitment(self, challenge: bytes, response):
        challenge = self._chal_from_bytes(challenge)
        morphism_of_response = self._morphism(response)
        return [phi_i - statement_i * Integer(challenge) 
                for (phi_i, statement_i) 
                in zip(morphism_of_response, self.statement)]

    def _morphism(self, x):
        raise NotImplementedError
        
    def _morphism_label(self):
        raise NotImplementedError
    
    def label(self):
        return self._morphism_label()
    
    def verifier(self, commitment, challenge: bytes, response):
        challenge = self._chal_from_bytes(challenge)
        return all(phi_response_i == commitment_i + statement_i * int(challenge)
            for phi_response_i, commitment_i, statement_i in zip(self._morphism(response), commitment, self.statement))
    
    def _chal_from_bytes(self, challenge: bytes) -> ec.Fp:
        with seed(hash_to_seed(challenge)):
            return self.ec.Fp.random_element()

In [222]:
class SchnorrDlog(DlogTemplate):
    ec = secp256k1
    n = 1
    m = 1
        
    first_label_hash = hash_function()
    first_label_hash.update(b"schnorr" + pickle.dumps(ec))
    
    # Inputs an array of one Fp element `x`
    # Outputs `[G * x]`
    
    def _morphism(self, x):
        return [self.ec.G * int(x[0])]
    
    def _morphism_label(self):
        label = hash_function()
        label.update(b"schnorr")
        label.update(pickle.dumps(self.ec.G))
        label.update(pickle.dumps(self.statement[0]))
        return label.digest()

Proof of knowledge of a secret key `sk0` for Schnorr Signature

In [223]:
sk0 = [SchnorrDlog.ec.Fp.random_element()]
pk0 = [SchnorrDlog.ec.G * Integer(sk0[0])]

schnorr0 = SchnorrDlog(b"context", pk0)
pstate, commitment = schnorr0.prover_commit(sk0)
challenge = random_bytes(32)
response = schnorr0.prover_response(pstate, challenge)
schnorr0.verifier(commitment, challenge, response)

True

In [224]:
batch_proof = schnorr0.batchable_proof(sk0, b"some message")
schnorr0.batchable_verify(batch_proof, b"some message")

True

In [225]:
short_proof = schnorr0.short_proof(sk0, b"a different message")
schnorr0.short_verify(short_proof, b"a different message")

True

`SigmaAndComposition` is used to prove knowledge of two independent witnesses. 

In [226]:
class SigmaAndComposition(SigmaProtocol):

    # Left and right are both sigma protocols
    def __init__(self, left: SigmaProtocol, right: SigmaProtocol):
        self.left = left
        self.right = right
        
        self.first_block_hash = self._get_first_block_hash(b"")
        
    def prover_commit(self, witness):
        w0, w1 = witness 
        left_state, left_commitment = self.left.prover_commit(w0)
        right_state, right_commitment = self.right.prover_commit(w1)
        return (left_state, right_state), (left_commitment, right_commitment)
    
    def prover_response(self, state, challenge):
        left_state, right_state = state
        left_response = self.left.prover_response(left_state, challenge)
        right_response = self.right.prover_response(right_state, challenge)
        return (left_response, right_response)
        
    def label(self):
        label = hash_function()
        label.update(pad_to_blocklen(b"zkpstd/sigma/and-v0.0.1"))
        label.update(self.left.label())
        label.update(self.right.label())
        return label.digest()
    
    def simulate_commitment(self, challenge, response):
        left_response, right_response = response
        left_commitment = self.left.simulate_commitment(challenge, left_response)
        right_commitment = self.right.simulate_commitment(challenge, right_response)
        return (left_commitment, right_commitment)
    
    def simulate_response(self):
        return (self.left.simulate_response(), self.right.simulate_response())

    def verifier(self, commitment, challenge, response):
        left_commitment, right_commitment = commitment
        left_response, right_response = response
        return (self.left.verifier(left_commitment, challenge, left_response)) and \
        (self.right.verifier(right_commitment, challenge, right_response))

An `AndComposition`, proving knowledge of two different secret keys `sk0` and `sk1`

In [227]:
sk1 = [SchnorrDlog.ec.Fp.random_element()]
pk1 = [G * Integer(sk1[0])]

schnorr1 = SchnorrDlog(b"context", pk1)
schnorrAnd = SigmaAndComposition(schnorr0, schnorr1)

and_proof = schnorrAnd.batchable_proof((sk0, sk1), b"message")
schnorrAnd.batchable_verify(and_proof, b"message")

True

Discrete Log Equality protocol example, also using the `secp256k1` curve. This is used for proving that $Y_1 = wG$ and $Y_2=wH$, where $G, H$ are generators of the `secp256k1` group. 

In [228]:
class DlogEQ(DlogTemplate):
    ec = secp256k1
    n = 2
    m = 1
    
    # Inputs an array of one Fp element `x`
    # Outputs `[G * x, H * x]`
    def _morphism(self, x):
        return [self.ec.G * int(x[0]), self.ec.H * int(x[0])]
    
    def _morphism_label(self):
        label = hash_function()
        label.update(b"dleq")
        label.update(pickle.dumps(self.ec.G))
        label.update(pickle.dumps(self.ec.H))
        label.update(pickle.dumps(self.statement[0]))
        label.update(pickle.dumps(self.statement[1]))
        return label.digest()

In [229]:
witness = [DlogEQ.ec.Fp.random_element()]
statement = [G * Integer(witness[0]), H * Integer(witness[0])]

dleq = DlogEQ(b"ctx", statement)
pstate, commitment = dleq.prover_commit(witness)
challenge = random_bytes(23)
response = dleq.prover_response(pstate, challenge)
dleq.verifier(commitment, challenge, response)

TypeError: 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp' object is not subscriptable